In [1]:
import shutil
import os

In [5]:
folder = '/tf/data/upd_face_dataset'

In [8]:
bad_people = [
    "aabarstok",
    "agmargaryan",
    "alalkhromov",
    "nepyrlitsanu",
    "avmatvienko_1",
    "aazheltov_1",
    "amsirotkin",
    "drsmirnov",
    "dvzaitseva",
    "eggadzhiev",
    "myumikheev",
    "rrminnegalieva",
    "vakabisov"
]

main_folders = ["up", "center", "down"]
subfolders = ["45_left", "center", "45_right"]

In [9]:
def copy_folders(folder, bad_people):
  upd_dataset = os.path.join(os.path.dirname(folder), "testt")
  os.makedirs(upd_dataset, exist_ok=True)
  for main_folder in main_folders:
    for subfolder in subfolders:
      subfolder_path = os.path.join(folder, main_folder, subfolder)
      for root, dirs, files in os.walk(subfolder_path):
        for dir in dirs:
          if dir.lower() not in bad_people:
            src_path = os.path.join(root, dir)
            new_path = os.path.join(upd_dataset, os.path.relpath(src_path, folder))
            try:
              shutil.copytree(src_path, new_path)
              print(f"copied: {src_path}")
            except Exception as e:
              print(f"{src_path} error: {e} ")

copy_folders(folder, bad_people)

copied: /tf/data/upd_face_dataset/up/45_left/aamartynenko_9
copied: /tf/data/upd_face_dataset/up/45_left/aazheltov_1
copied: /tf/data/upd_face_dataset/up/45_left/adletunov
copied: /tf/data/upd_face_dataset/up/45_left/adpiskunov
copied: /tf/data/upd_face_dataset/up/45_left/aimilovanova
copied: /tf/data/upd_face_dataset/up/45_left/ameliseenko
copied: /tf/data/upd_face_dataset/up/45_left/amsirotkin
copied: /tf/data/upd_face_dataset/up/45_left/amzayakina
copied: /tf/data/upd_face_dataset/up/45_left/appecherskih
copied: /tf/data/upd_face_dataset/up/45_left/arandarbua
copied: /tf/data/upd_face_dataset/up/45_left/archilingarov
copied: /tf/data/upd_face_dataset/up/45_left/arkhabibullina_3
copied: /tf/data/upd_face_dataset/up/45_left/asbaranova
copied: /tf/data/upd_face_dataset/up/45_left/avpapina
copied: /tf/data/upd_face_dataset/up/45_left/ayuvasileva_1
copied: /tf/data/upd_face_dataset/up/45_left/dashubin
copied: /tf/data/upd_face_dataset/up/45_left/datikhomirov
copied: /tf/data/upd_face_dat

In [11]:
!pip install torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 13.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 6.7 MB/s eta 0:00:00:00:0100:02
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1
    Uninstalling torch-2.2.1:
      Successfully uninstalled torch-2.2.1


In [12]:
from torchvision.utils import make_grid, save_image
from torchvision.transforms import transforms, functional
from torchvision import torch
from PIL import Image
import os
from tqdm import tqdm

In [13]:
dataset_dir = '/tf/data/testt'
# user_name = "prybakov"
# user_name = "aabarstok"
horizontal_positions = ["45_left", "center", "45_right"]
vertical_positions = ["up", "center", "down"]
# users_list = os.listdir(r'C:\Users\HYPERPC\Documents\miem-project-1712\upd_face_dataset\center\center')
users_list = os.listdir('/tf/data/testt/center/center')
save_dir = os.path.join(dataset_dir, "grids")

# check if saving directory does exist
isExist = os.path.exists(save_dir)
if not isExist:
    os.makedirs(save_dir, exist_ok=True)

transform = transforms.Compose([
    transforms.ConvertImageDtype(dtype=torch.float),
    transforms.Resize((600, 400), antialias=True),
])


def save_grid_image(tensors, glasses):
    grid = make_grid(tensors, nrow=3, padding=1)
    img = transforms.ToPILImage()(grid)
    # img.show()
    hpercent = (1080 / float(img.size[1]))
    wsize = int((float(img.size[0]) * float(hpercent)))
    img = img.resize((wsize, 1080), Image.LANCZOS)
    save_image(grid, os.path.join(save_dir, f"{user_name}_grid_{glasses}.jpg"))


for user_name in tqdm(users_list):
    tensors_glasses = []
    tensors_no_glasses = []
    image_glasses = os.path.join(dataset_dir,
                                 f"{vertical_positions[0]}/{horizontal_positions[0]}/{user_name}/glasses.JPG")
    image_no_glasses = os.path.join(dataset_dir,
                                    f"{vertical_positions[0]}/{horizontal_positions[0]}/{user_name}/no_glasses.JPG")
    yesGlasses = os.path.exists(image_glasses)
    noGlasses = os.path.exists(image_no_glasses)
    if noGlasses:
        if (os.path.exists(os.path.join(save_dir, f"{user_name}_grid_no_glasses.jpg"))):
            continue
        try:
            for vertical_position in vertical_positions:
             for horizontal_position in horizontal_positions:
                image_no_glasses = os.path.join(dataset_dir,
                                                f"{vertical_position}/{horizontal_position}/{user_name}/no_glasses.JPG")
                # PIL respects image metadata and since our camera was rotated during shoot all images are opened rotated and have to be rotated back
                transformed_tensor = transform(
                    functional.pil_to_tensor(Image.open(image_no_glasses).transpose(Image.ROTATE_90)))
                # transformed_tensor = transform(functional.pil_to_tensor(Image.open(image)))
                tensors_no_glasses.append(transformed_tensor)
                save_grid_image(tensors_no_glasses, "no_glasses")
        except FileNotFoundError:
            print(f"No no_glasses images for {user_name}")

    if yesGlasses:
        if (os.path.exists(os.path.join(save_dir, f"{user_name}_grid_glasses.jpg"))):
            continue
        try:
            for vertical_position in vertical_positions:
             for horizontal_position in horizontal_positions:
                image_glasses = os.path.join(dataset_dir,
                                             f"{vertical_position}/{horizontal_position}/{user_name}/glasses.JPG")
                # PIL respects image metadata and since our camera was rotated during shoot all images are opened rotated and have to be rotated back
                transformed_tensor = transform(
                    functional.pil_to_tensor(Image.open(image_glasses).transpose(Image.ROTATE_90)))
                # transformed_tensor = transform(functional.pil_to_tensor(Image.open(image)))
                tensors_glasses.append(transformed_tensor)
                save_grid_image(tensors_glasses, "glasses")
        except FileNotFoundError:
            print(f"No glasses images for {user_name}")

100%|██████████| 58/58 [03:02<00:00,  3.15s/it]


In [20]:
!pip install deepface

In [21]:
import glob
from deepface import DeepFace

2024-04-18 12:35:28.153051: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-18 12:35:28.153109: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-18 12:35:28.153158: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-18 12:35:28.159389: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [26]:
def faces_height(face_info):
  return face_info["facial_area"]["h"]

In [27]:
folder = '/tf/data/testt/center/center'

In [28]:
files = glob.glob(os.path.join(folder, "**", "*.JPG"), recursive=True)

In [30]:
def height_f(folder):
    for img_path in files:
        face_objs = DeepFace.extract_faces(img_path=img_path, detector_backend='retinaface')
        if face_objs:
            face_height = faces_height(face_objs[0])
            if face_height < 1024:
                print(f"{img_path}: {face_height}")

height_f(folder)

/tf/data/testt/center/center/aazheltov_1/no_glasses.JPG: 807
/tf/data/testt/center/center/amsirotkin/no_glasses.JPG: 830
/tf/data/testt/center/center/drsmirnov/glasses.JPG: 973
/tf/data/testt/center/center/drsmirnov/no_glasses.JPG: 940
/tf/data/testt/center/center/dvzaitseva/glasses.JPG: 755
/tf/data/testt/center/center/dvzaitseva/no_glasses.JPG: 771
/tf/data/testt/center/center/eggadzhiev/glasses.JPG: 655
/tf/data/testt/center/center/eggadzhiev/no_glasses.JPG: 672
/tf/data/testt/center/center/myumikheev/no_glasses.JPG: 973
/tf/data/testt/center/center/rrminnegalieva/glasses.JPG: 1016
/tf/data/testt/center/center/vakabisov/no_glasses.JPG: 877
